## Model:QA Calls For Evaluation
Logging every call to the chain for later review/analysis

In [1]:
%pip install langchain langsmith -y
%pip cache purge

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install [options] <archive url/path> ...

no such option: -y


Files removed: 166 (92.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install optimum

Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
from langchain import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from dotenv import load_dotenv
import os
import openai

## Set Up Dataset

In [2]:
audio_test_set = [
    {"question": "What is ServiceNow?", "gold_answer": "ServiceNow is a cloud platform.", "audio_file": "audio1.wav", "gold_transcript": "ServiceNow is a cloud platform."},
    ...
]

 ## Define Pipelines

In [2]:
from transformers import pipeline

hf_pipes = {
    "distilgpt2": pipeline("text-generation", model="distilgpt2"),
    "flan-t5-base": pipeline("text2text-generation", model="google/flan-t5-base"),
    "flan-t5-large": pipeline("text2text-generation", model="google/flan-t5-large"),
    "mistral-7b": pipeline("text-generation", model="TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"),
}

# 2. Define OpenAI function
def ask_gpt3_turbo(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=[{"role":"user","content":prompt}]
    )
    return response.choices[0].message.content.strip()

Device set to use cpu
Device set to use cpu
Device set to use cpu


ValueError: Could not load model TheBloke/Mistral-7B-Instruct-v0.2-GPTQ with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.mistral.modeling_mistral.MistralForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\transformers\pipelines\base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\transformers\models\auto\auto_factory.py", line 600, in from_pretrained
    return model_class.from_pretrained(
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-packages\transformers\modeling_utils.py", line 311, in _wrapper
    return func(*args, **kwargs)
  File "c:\Users\Mercy\.pyenv\pyenv-win\versions\3.10.11\lib\site-pa

In [ ]:
whisper_asr = pipeline("automatic-speech-recognition", model="openai/whisper-base")

## Evaluation Loop

In [ ]:
results = []

# Use audio_test_set as test_set if it contains the required fields
test_set = audio_test_set

for example in test_set:  # test_set: list of dicts
    q = example['question']
    gold = example['gold_answer']
    row = {'question': q, 'gold_answer': gold}

    # Hugging Face models
    for name, pipe in hf_pipes.items():
        if 't5' in name:  # text2text
            out = pipe(q)[0]['generated_text']
        else:  # text-generation
            out = pipe(q, max_new_tokens=64)[0]['generated_text']
        row[name] = out

    # OpenAI
    row["gpt-3.5-turbo"] = ask_gpt3_turbo(q)
    results.append(row)


In [ ]:
for example in audio_test_set:
    audio_path = example['audio_file']
    gold = example['gold_transcript']
    asr_out = whisper_asr(audio_path)['text']

## Calculate Metrics

In [ ]:
from sklearn.metrics import f1_score
def token_f1(a, b):
    a, b = set(a.lower().split()), set(b.lower().split())
    if not a or not b: return 0
    common = a & b
    precision = len(common)/len(a)
    recall = len(common)/len(b)
    return 2*precision*recall/(precision+recall+1e-8)

for r in results:
    for model in hf_pipes.keys() + ["gpt-3.5-turbo"]:
        r[f'{model}_f1'] = token_f1(r[model], r['gold_answer'])


In [ ]:
import jiwer  # pip install jiwer
for example in audio_test_set:
    asr_out = whisper_asr(example['audio_file'])['text']
    example['wer'] = jiwer.wer(example['gold_transcript'], asr_out)

## Compare Results

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
print(df[[f'{model}_f1' for model in hf_pipes.keys() + ["gpt-3.5-turbo"]]].mean())

In [ ]:
asr_df = pd.DataFrame(audio_test_set)
print(asr_df['wer'].mean())